In [ ]:
import os
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate#PromptTemplate LangChain me ek helper class hai jo LLM ke liye prompts 
#ko dynamically generate karne ke liye use hota hai.
from langchain_core.documents import Document#from langchain_core.documents import Document LangChain me ek bahut important class hai, 
#jo text + metadata ko represent karne ke liye use hota hai.
from langchain_core.runnables import  #RunnablePassthrough LangChain 2024+ (LCEL) ka ek powerful utility hai, 
#jo input ko bina badle aage pass kar deta hai.

# 1. LLM Load 
llm = ChatOllama(model="llama3.1")
google_api_key=os.environ["GOOGLE_API_KEY"]


# 2. Dummy Docs
docs = [Document(page_content="LangChain helps build chatbots using LLMs and retrieval.")]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
splits = text_splitter.split_documents(docs)

# 3. Embeddings + Chroma
embeddings = OllamaEmbeddings(model="nomic-embed-text")

vectordb = Chroma.from_documents(splits, embedding=embeddings)

# 4. Retriever
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

# 5. Prompt
prompt_template = """
You are a helpful AI assistant.
Answer the question using ONLY the given context.

Context:
{context}

Question:
{question}

Answer:
"""
prompt = PromptTemplate.from_template(prompt_template)

# 6. NEW RAG PIPELINE (RunnableGraph)
def format_docs(docs):#Retriever se list of Document objects aate hain , Aap unka sirf text extract karke join kar rahe ho
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "context": retriever | format_docs,#Left side (context) → pehle retriever chalega → fir format_docs → result prompt me jayega
#Right side (question) → input as-is (same user query) pass hoga prompt ko
        "question": RunnablePassthrough()
    }
    | prompt  #Template variables {context} & {question} fill honge, Final prompt LLM ko bheja jayega
    | llm
)

# 7. Ask Question
query = "What is chain in langchain?"
response = rag_chain.invoke(query)

print("Answer:", response.content) 

#In LangChain, a Document is a structured container that stores text (page_content) along with optional metadata.
# It is used in RAG pipelines for splitting, embedding, storing, and retrieving context.

#RunnablePassthrough is a LangChain LCEL utility that forwards the input unchanged to the next step. 
#It is essential when building multi-branch pipelines where part of the input needs preprocessing 
#while another part must pass through untouched.

#“Ek reusable template jo aapke LLM prompt ko dynamic banata hai, jisme aap variables inject kar sakte ho jaise {question} aur {context}. 
# Isse LLM ko har query ke liye customized prompt bhejna easy ho jata hai.

c:\simple_chatbot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Answer: Based on the provided context, I believe the "chain" in LangChain refers to a method of combining multiple Language Models (LLMs) or other information sources into a single, cohesive system. The term "chain" likely indicates that these models or sources are being linked together to create a more powerful and robust chatbot.

In this sense, the chain might represent a sequence or network of LLMs and retrieval systems working in concert to provide more accurate and comprehensive responses to user queries. This could involve using one model to retrieve relevant information, another to generate text based on that information, and yet another to refine or correct the output.
